In [2]:
import os, sys
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

from src.configuration.constants import PROCESSED_DATA_DIRECTORY, ROOT_DIRECTORY, INTERIM_DATA_DIRECTORY
from src.utils.dataset import load_dataset

import logging
import random
import numpy as np
import pickle
from numpy import expand_dims, zeros, ones, asarray
from numpy.random import randn, randint

from matplotlib import pyplot

import tensorflow as tf
from functools import partial
from mlprimitives import load_primitive

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from orion.evaluation.contextual import contextual_f1_score
import pandas as pd
import random


from mlprimitives import load_primitive

logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

https://machinelearningmastery.com/semi-supervised-generative-adversarial-network/

# Phase 0: Prepare dataset

In [3]:
dataset = load_dataset('interim', 'SMAP')

train_split = dataset['train'][dataset['train'].signal == 'A3Benchmark-TS12']
test_split = dataset['test'][dataset['test'].signal == 'A3Benchmark-TS12']
anomalies_split = dataset['anomaly'][dataset['anomaly'].signal == 'A3Benchmark-TS12']

In [72]:
test_split.anomaly.sum()

5

In [47]:
index_train = train_split['index'].astype(int)
X_train = train_split[['anomaly'] + list(train_split.columns)[5:]]

index_test = test_split['index'].astype(int).reset_index(drop=True)
X_test = test_split[['anomaly'] + list(test_split.columns)[5:]]

In [48]:
def fit_processing(X, index):
    primitives = []
    
    
    # This primitive is an imputation transformer for filling missing values
    params = {
        'X': X
    }
    primitive = load_primitive('sklearn.impute.SimpleImputer', arguments=params)
    primitive.fit()
    primitives.append(primitive)
    X = primitive.produce(X=X)
    print(primitive, X.shape)
    
    # This primitive transforms features by scaling each feature to a given range
    params = {
        "feature_range": [-1, 1], 
        'X': X,
    }
    primitive = load_primitive('sklearn.preprocessing.MinMaxScaler', arguments=params)
    primitive.fit()
    primitives.append(primitive)
    X = primitive.produce(X=X)
    print(primitive, X.shape)
    
    # Uses a rolling window approach to create the sub-sequences out of time series data
    params = {
        "target_column": 0, 
        "window_size": 100, 
        'target_size': 1, 
        'step_size': 1
    }
    primitive = load_primitive('mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences',
                               arguments=params)
    primitives.append(primitive)
    X, y, index, target_index = primitive.produce(X=X, index=index)

    # Target / target size is the next interval that is trying to predict.
    # Index is the start of the interval
    print(primitive, X.shape, y.shape, index.shape, target_index.shape)
    
    return X, index, primitives
    
    

In [49]:
X, index, primitives = fit_processing(X_train, index_train)

MLBlock - sklearn.impute.SimpleImputer (1120, 8)
MLBlock - sklearn.preprocessing.MinMaxScaler (1120, 8)
MLBlock - mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences (1020, 100, 8) (1020, 1) (1020,) (1020,)


In [50]:
X_train = X[:, :, 1:]
y_train = np.expand_dims(X_train[:, :, 0], 2)
index_train = index
labels_train = np.array([1 if sum(i) > 0 else 0 for i in X[:, :, 0]])

X_train.shape, y_train.shape, index_train.shape, labels_train.shape

((1020, 100, 7), (1020, 100, 1), (1020,), (1020,))

In [51]:
def produce_processing(X, index, primitives):
    X = primitives[0].produce(X=X)
    X = primitives[1].produce(X=X)
    X, y, index, target_index = primitives[2].produce(X=X, index=index)
    return X, index

In [52]:
X, index = produce_processing(X_test, index_test, primitives)

In [53]:
X_test = X[:, :, 1:]
y_test = np.expand_dims(X_test[:, :, 0], 2)
index_test = index
labels_test = np.array([1 if sum(i) > 0 else 0 for i in X[:, :, 0]])


X_test.shape, y_test.shape, index_test.shape, labels_test.shape

((460, 100, 7), (460, 100, 1), (460,), (460,))

# Phase 1: Semi-supervised GAN

## Discriminator

### Separate Discriminator Models With Shared Weights


#### Encoder

In [54]:
def build_encoder(input_shape, lstm_units, dense_units, encoder_reshape_shape):
    x = Input(shape=input_shape)
    model = Sequential()
    model.add(Bidirectional(LSTM(units=lstm_units, return_sequences=True)))
    model.add(Flatten())
    model.add(Dense(units=dense_units))
    model.add(Reshape(target_shape=encoder_reshape_shape))
    return Model(x, model(x))

#### Generator

In [55]:
def build_generator(input_shape, generator_reshape_dim, generator_reshape_shape):
    x = Input(shape=input_shape)
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(units=generator_reshape_dim))
    model.add(Reshape(target_shape=generator_reshape_shape))
    model.add(Bidirectional(LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    model.add(UpSampling1D(size=2))
    model.add(Bidirectional(LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    model.add(TimeDistributed(Dense(units=1)))
    model.add(Activation(activation='tanh'))
    return Model(x, model(x))
              

#### Critic X

In [56]:
def build_critic_x(input_shape, n_classes):
    x = Input(shape=input_shape)
    model = Sequential()
    
    for _ in range(4):
        model.add(Conv1D(filters=64, kernel_size=5))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(rate=0.25))
    
    model.add(Flatten())
    
    # Unsupervised model
    unsup_out = Dense(1, activation='sigmoid')(model(x))
    d_unsup_model = Model(x, unsup_out)
    
    # Supervised model
    sup_out = Dense(n_classes, activation='softmax')(model(x))
    d_sup_model = Model(x, sup_out)
    
    return d_unsup_model, d_sup_model

#### Critic Z

In [57]:
def build_critic_z(input_shape, dense_units=20):
    x = Input(shape=input_shape)
    model = Sequential()
    model.add(Flatten())
    
    for _ in range(2):
        model.add(Dense(units=dense_units))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(rate=0.2))
        
    model.add(Dense(units=1))
    return Model(x, model(x))

#### Hyper-parameters

In [59]:
input_shape=X_test[0].shape
target_shape=y_test[0].shape
latent_dim=20
learning_rate=0.0005
epochs=20
batch_size=64
iterations_critic=5
latent_shape = (latent_dim, 1)
n_classes = 2

shape = np.asarray(X_test)[0].shape
length = shape[0]
target_shape = np.asarray(y_test)[0].shape


generator_reshape_dim = length // 2
generator_reshape_shape = (length // 2, 1)
encoder_reshape_shape = latent_shape

encoder_input_shape = shape
generator_input_shape = latent_shape
critic_x_input_shape = target_shape
critic_z_input_shape = latent_shape

lstm_units = 100
dense_units = 20

optimizer = tf.keras.optimizers.Adam(learning_rate)

In [60]:
encoder = build_encoder(
    input_shape=encoder_input_shape,
    lstm_units=lstm_units,
    dense_units=dense_units,
    encoder_reshape_shape=encoder_reshape_shape,
)

generator = build_generator(
    input_shape=generator_input_shape,
    generator_reshape_dim=generator_reshape_dim,
    generator_reshape_shape=generator_reshape_shape,
)

critic_x_unsup, critic_x_sup = build_critic_x(
    input_shape=critic_x_input_shape,
    n_classes=n_classes
)

critic_z = build_critic_z(
    input_shape=critic_z_input_shape,
)

In [61]:
def _wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

def _gradient_penalty_loss(y_true, y_pred, averaged_samples):
    gradients = K.gradients(y_pred, averaged_samples)[0]
    gradients_sqr = K.square(gradients)
    gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    gradient_penalty = K.square(1 - gradient_l2_norm)
    return K.mean(gradient_penalty)

In [62]:
generator.trainable = False
encoder.trainable = False

x = Input(shape=input_shape)
y = Input(shape=target_shape)
z = Input(shape=(latent_dim, 1))

x_ = generator(z)
z_ = encoder(x)
fake_x = critic_x_unsup(x_) # Fake
valid_x = critic_x_unsup(y) # Truth

label = critic_x_sup(y)

#### Critic X Unsupervised Model

In [63]:
alpha = K.random_uniform((64, 1, 1))
interpolated_x = (alpha * [y, x_][0]) + ((1 - alpha) * [y, x_][1])
validity_interpolated_x = critic_x_unsup(interpolated_x)
partial_gp_loss_x = partial(_gradient_penalty_loss, averaged_samples=interpolated_x)
partial_gp_loss_x.__name__ = 'gradient_penalty'
critic_x_unsup_model = Model(inputs=[y, z], outputs=[valid_x, fake_x,validity_interpolated_x])
critic_x_unsup_model.compile(loss=[_wasserstein_loss, _wasserstein_loss, partial_gp_loss_x], 
                       optimizer=optimizer, loss_weights=[1, 1, 10])
critic_x_unsup_model.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 20, 1)]      0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
model_9 (Model)                 (None, 100, 1)       133787      input_16[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_mul_4 (TensorFlowOp [(64, 100, 1)]       0           input_15[0][0]                   
___________________________________________________________________________________________

#### Critic X Supervised Model

In [64]:
critic_x_sup.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=optimizer, 
    metrics=['accuracy'], # f1, precision, recall
)
critic_x_sup_model = critic_x_sup
critic_x_sup_model.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 100, 1)]          0         
_________________________________________________________________
sequential_6 (Sequential)    multiple                  62016     
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 10754     
Total params: 72,770
Trainable params: 72,770
Non-trainable params: 0
_________________________________________________________________


#### Critic Z Model

In [65]:
fake_z = critic_z(z_)
valid_z = critic_z(z)
alpha = K.random_uniform((64, 1, 1))
interpolated_z = (alpha * [z, z_][0]) + ((1 - alpha) * [z, z_][1])
validity_interpolated_z = critic_z(interpolated_z)
partial_gp_loss_z = partial(_gradient_penalty_loss, averaged_samples=interpolated_z)
partial_gp_loss_z.__name__ = 'gradient_penalty'
critic_z_model = tf.keras.Model(inputs=[x, z], outputs=[valid_z, fake_z,validity_interpolated_z])
critic_z_model.compile(loss=[_wasserstein_loss, _wasserstein_loss,
                                  partial_gp_loss_z], optimizer=optimizer,
                            loss_weights=[1, 1, 10])
critic_z_model.summary()

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 100, 7)]     0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, 20, 1)]      0                                            
__________________________________________________________________________________________________
model_8 (Model)                 (None, 20, 1)        486420      input_14[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_mul_6 (TensorFlowOp [(64, 20, 1)]        0           input_16[0][0]                   
___________________________________________________________________________________________

#### Encoder Generator Model

In [66]:
critic_x_sup.trainable = False
critic_x_unsup.trainable = False
critic_z.trainable = False
generator.trainable = True
encoder.trainable = True

z_gen = Input(shape=(latent_dim, 1))
x_gen_ = generator(z_gen)
x_gen = Input(shape=input_shape)
z_gen_ = encoder(x_gen)
x_gen_rec = generator(z_gen_)
fake_gen_x = critic_x_unsup(x_gen_)
fake_gen_z = critic_z(z_gen_)

encoder_generator_model = Model([x_gen, z_gen], [fake_gen_x, fake_gen_z, x_gen_rec])
encoder_generator_model.compile(loss=[_wasserstein_loss, _wasserstein_loss,'mse'], 
                                optimizer=optimizer,
                                loss_weights=[1, 1, 10])

encoder_generator_model.summary()

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 20, 1)]      0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 100, 7)]     0                                            
__________________________________________________________________________________________________
model_9 (Model)                 (None, 100, 1)       133787      input_17[0][0]                   
                                                                 model_8[2][0]                    
__________________________________________________________________________________________________
model_8 (Model)                 (None, 20, 1)        486420      input_18[0][0]            

## Dataset Selection

In [67]:
def split_supervised_samples(y, labels, limit=None):
    labels = pd.Series(labels)
    
    sup_anomaly_index = labels[labels==1].index
    if limit is not None:
        sup_anomaly_index = random.choices(sup_anomaly_index, k=limit)
        
    sup_anomaly_samples = y[sup_anomaly_index]
    num_anomaly_samples = len(sup_anomaly_samples)
    
    sup_non_anomaly_index = random.choices(labels[labels==0].index,  k=num_anomaly_samples)
    sup_non_anomaly_samples = y[sup_non_anomaly_index]
    
    sup_samples = np.concatenate((sup_anomaly_samples, sup_non_anomaly_samples))
    sup_labels = np.concatenate((np.ones(len(sup_anomaly_samples)), np.zeros(len(sup_non_anomaly_samples))))
    
    unsup_samples = y[~np.isin(np.arange(len(y)), list(sup_anomaly_index) + list(sup_non_anomaly_index))]
    
    return sup_samples, sup_labels, unsup_samples

In [68]:
sup_samples, sup_labels, unsup_samples = split_supervised_samples(y_test, labels_test, 0)

sup_samples.shape, sup_labels.shape, unsup_samples.shape

((0, 100, 1), (0,), (460, 100, 1))

In [69]:
def select_supervised_samples(n_samples, X, labels, n_classes):
    """Select a supervised subset of the dataset, ensures classes are balanced."""
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)
    for i in range(n_classes):
        X_with_class = X[labels == i]
        ix = randint(0, len(X_with_class), n_per_class)
        [X_list.append(X_with_class[j]) for j in ix]
        [y_list.append(i) for j in ix]
    return asarray(X_list), asarray(y_list)

def generate_real_samples(n_samples, X, labels):
    """Randomly select real samples."""
    ix = randint(0, X.shape[0], n_samples)
    X, labels = X[ix], labels[ix]
    y = ones((n_samples, 1))
    return [X, labels], y

def generate_latent_points(n_samples, latent_dim):
    """Generate points in latent space as input for the generator."""
    return np.random.normal(size=(n_samples, latent_dim, 1))
 
def generate_fake_samples(n_samples, latent_dim, generator):
    """Generate n fake examples with class labels."""
    z = generate_latent_points(n_samples, latent_dim)
    x_ = generator(z)
    y = zeros((n_samples, 1))
    return x_, y

## Training

In [93]:
enable_supervised = None

fake = np.ones((batch_size, 1))
valid = -np.ones((batch_size, 1))
delta = np.ones((batch_size, 1))

sup_samples, sup_labels, unsup_samples = split_supervised_samples(y_train, labels_train, 1)


indices = np.arange(X_train.shape[0])
for epoch in range(1, epochs + 1):
    np.random.shuffle(indices)
    X_ = X_train[indices]
    y_ = y_train[indices]

    epoch_g_loss = []
    epoch_cx_loss = []
    epoch_cz_loss = []
    epoch_cx_sup_loss = []

    minibatches_size = batch_size * iterations_critic
    num_minibatches = int(X_.shape[0] // minibatches_size)

    for i in range(num_minibatches):
        minibatch = X_[i * minibatches_size: (i + 1) * minibatches_size]
        y_minibatch = y_[i * minibatches_size: (i + 1) * minibatches_size]
        
        if enable_supervised:
            [X_sup_real, y_sup_real], _ = generate_real_samples(minibatches_size, sup_samples, sup_labels)
            epoch_cx_sup_loss.append(
                critic_x_sup_model.train_on_batch(X_sup_real, y_sup_real)
            )

        for j in range(iterations_critic):
            x = minibatch[j * batch_size: (j + 1) * batch_size]
            y = y_minibatch[j * batch_size: (j + 1) * batch_size]
            z = np.random.normal(size=(batch_size, latent_dim, 1))
            
            epoch_cx_loss.append(
                critic_x_unsup_model.train_on_batch([y, z], [valid, fake, delta]))
            epoch_cz_loss.append(
                critic_z_model.train_on_batch([x, z], [valid, fake, delta]))

        epoch_g_loss.append(
            encoder_generator_model.train_on_batch([x, z], [valid, valid, y]))

    cx_loss = np.mean(np.array(epoch_cx_loss), axis=0)
    cz_loss = np.mean(np.array(epoch_cz_loss), axis=0)
    g_loss = np.mean(np.array(epoch_g_loss), axis=0)
    cx_sup_loss = np.mean(np.array(epoch_cx_sup_loss), axis=0)
    
    print('Epoch: {}/{}, [Dx loss: {}] [Dx_sup loss: {}] [Dz loss: {}] [G loss: {}]'.format(
        epoch, epochs, cx_loss, cx_sup_loss, cz_loss, g_loss))

IndexError: index 0 is out of bounds for axis 0 with size 0

## Evaluation of unsupervised model

In [77]:
z_ = encoder.predict(X_test)
y_hat = generator.predict(z_)
critic = critic_x_unsup.predict(y_test)

z_.shape, y_hat.shape, critic.shape

((460, 20, 1), (460, 100, 1), (460, 1))

In [78]:
# computes an array of anomaly scores based on a combination of reconstruction error and critic output
params = {"rec_error_type": "dtw"}

primitive = load_primitive("orion.primitives.tadgan.score_anomalies", arguments=params)
errors, true_index, true, predictions = primitive.produce(y=y_test, y_hat=y_hat, critic=critic, index=index_train)

errors.shape, true_index.shape

((559,), (1020,))

In [88]:
# extracts anomalies from sequences of errors following the approach
params = {
    "window_size_portion": 0.1, 
    "window_step_size_portion": 0.1,
    "fixed_threshold": True
}

primitive = load_primitive("orion.primitives.timeseries_anomalies.find_anomalies", 
                           arguments=params)
e = primitive.produce(errors=errors, index=true_index)

e.shape

(1, 3)

In [89]:
ground_truth

,source,name,signal,start,end
0,Yahoo,A3,A3Benchmark-TS12,1.416766e+09,1.416766e+09
1,Yahoo,A3,A3Benchmark-TS12,1.418414e+09,1.418414e+09
2,Yahoo,A3,A3Benchmark-TS12,1.418627e+09,1.418627e+09
3,Yahoo,A3,A3Benchmark-TS12,1.418670e+09,1.418670e+09
4,Yahoo,A3,A3Benchmark-TS12,1.419592e+09,1.419592e+09
5,Yahoo,A3,A3Benchmark-TS12,1.420009e+09,1.420009e+09
6,Yahoo,A3,A3Benchmark-TS12,1.420200e+09,1.420200e+09
7,Yahoo,A3,A3Benchmark-TS12,1.420571e+09,1.420571e+09
8,Yahoo,A3,A3Benchmark-TS12,1.420992e+09,1.420992e+09
9,Yahoo,A3,A3Benchmark-TS12,1.421039e+09,1.421039e+09


In [90]:
ground_truth = anomalies_split
anomalies = [(int(i[0]), int(i[1])) for i in e]
start, end = index_train[0], index_train[-1]
contextual_f1_score(ground_truth, anomalies, start=start, end=end, weighted=True)

1.1110246980790383e-05

In [31]:
0, 0.07946026986506748

100, 0.09651076466221233

300, 0.08085430968726164



(300, 0.08085430968726164)

# Phase 2: Query Strategies

In [193]:
# What did the unsupervised GAN say was an anomaly?
z_ = encoder.predict(X_train)
y_hat = generator.predict(z_)
critic = critic_x_unsup.predict(y_train)

params = {"rec_error_type": "dtw"}

primitive = load_primitive("orion.primitives.tadgan.score_anomalies", arguments=params)
errors, true_index, true, predictions = primitive.produce(y=y_train, y_hat=y_hat, critic=critic, index=index_train)

params = {
    "window_size_portion": 0.05, 
    "window_step_size_portion": 0.1,
    "fixed_threshold": True
}

primitive = load_primitive("orion.primitives.timeseries_anomalies.find_anomalies", 
                           arguments=params)
e = primitive.produce(errors=errors, index=true_index)

anomalies = [(int(i[0]), int(i[1])) for i in e]
pp_samples = []

for s, e in anomalies:
    for i in range(s, e):
        pp_samples.append(X_sup[i])

In [200]:
pp_samples = pd.Series(pp_samples)
pp_samples.shape

(0,)

In [188]:
# What did the supervised GAN get wrong?
X_sup, labels_sup
labels_sup = pd.Series(labels_sup)

sup_predictions = critic_x_sup_model.predict(X_sup)

idx = np.argmax(sup_predictions, axis=-1)
sup_predictions = np.zeros( sup_predictions.shape )
sup_predictions[ np.arange(sup_predictions.shape[0]), idx] = 1
sup_predictions = [list(i).index(1.0) for i in sup_predictions]
sup_predictions = pd.Series(sup_predictions)

# False Positives
fp = X_sup[(sup_predictions == 1) & (labels_sup == 0)]
fn = X_sup[(sup_predictions == 0) & (labels_sup == 1)]


In [196]:
fp

array([], shape=(0, 100, 1), dtype=float64)

In [189]:
# How should we explore datasets?

exploration = X_train[np.random.choice([i for i in range(len(X_train))], 20)]

In [201]:
annotation_set = [
    # pp_samples,
    fp,
    fn,
    exploration
]
annotation_set = np.concatenate(annotation_set)
annotation_set.shape

(26, 100, 1)

# Phase 3: Annotation Strategy

In [204]:
# How do we combine labels from different people?

labels = labels_train(annotation_set.index)

AttributeError: 'numpy.ndarray' object has no attribute 'index'